In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['CTR_CATEGO_X'] = train['CTR_CATEGO_X'].replace('N', 'M')
train['CTR_CATEGO_X'].value_counts()

M    12820
C     6835
P     5398
Name: CTR_CATEGO_X, dtype: int64

In [7]:
train['CTR_CATEGO_X'] = train['CTR_CATEGO_X'].replace('M', 1)
train['CTR_CATEGO_X'] = train['CTR_CATEGO_X'].replace('M', 2)
train['CTR_CATEGO_X'] = train['CTR_CATEGO_X'].replace('M', 3)

test['CTR_CATEGO_X'] = test['CTR_CATEGO_X'].replace('M', 1)
test['CTR_CATEGO_X'] = test['CTR_CATEGO_X'].replace('M', 2)
test['CTR_CATEGO_X'] = test['CTR_CATEGO_X'].replace('M', 3)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def sum_missing_values(data):
    cols = list(data.columns)
    
    for col in cols:
        print(f'{col}: {data[col].isna().sum()}')
        
train1 = train.copy()
test1 = test.copy()


for col in list(train1.columns):
    if train1[col].isna().sum() > 17000:
        train1 = train1.drop(col, axis=1)
        
for col in list(test1.columns):
    if test[col].isna().sum() > 3000:
        test1 = test1.drop(col, axis=1)
        
train1 = train1.drop('id', axis=1)
test1 = test1.drop('id', axis=1)

#train['target'] = train['target'].replace(0.0, 0.01)
from sklearn.preprocessing import StandardScaler

labels = np.array(train['target']).reshape(-1,1)
scalery = StandardScaler().fit(labels)
labels = scalery.transform(labels)

train1 = train1.drop('target', axis=1)

def fill_mean(data):
    cols = list(data.columns)
    
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)
        
    return data

train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('N', 'M')

train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('M', 1)
train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('C', 2)
train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('P', 3)


test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('M', 1)
test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('C', 2)
test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('P', 3)


train2 = train1.copy()
test2 = test1.copy()

#train2 = train2.drop('CTR_CATEGO_X', axis=1)
#test2 = test2.drop('CTR_CATEGO_X', axis=1)

tcrain2 = fill_mean(train2)
test2 = fill_mean(test2)

train_enc = np.array(train2)
test_enc = np.array(test2)

In [3]:
train_enc.shape, test_enc.shape

((25053, 89), (10738, 89))

In [3]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(train_enc.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = train_enc[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = train_enc[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [4]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [6]:
xg = xgb.XGBRegressor(boosting_type='gbtree', 
                      learning_rate=0.01, 
                      n_estimators=5000, 
                      max_depth=12, 
                      base_score=0.7,
                      max_features='auto'
                     )
eval_set = [(x_train, y_train), (x_test2, y_test2)]
xg.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=200)

[09:07:52] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type, max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:1.21870	validation_1-rmse:1.19890
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:1.21104	validation_1-rmse:1.19273
[2]	validation_0-rmse:1.20346	validation_1-rmse:1.18661
[3]	validation_0-rmse:1.19598	validation_1-rmse:1.18056
[4]	validation_0-rmse:1.18862	validation_1-rmse:1.17463
[5]	validation_0-rmse:1.18135	validation_1-rmse:1.16877
[6]	validation_0-rmse:1.17423	validation_1-rmse:1.16297
[7]	validation_0-rmse:1.16720	validation_1-rmse:1.15722
[8]	validation_0-rmse:1.16018	validat

[135]	validation_0-rmse:0.68134	validation_1-rmse:0.85206
[136]	validation_0-rmse:0.67946	validation_1-rmse:0.85128
[137]	validation_0-rmse:0.67759	validation_1-rmse:0.85056
[138]	validation_0-rmse:0.67607	validation_1-rmse:0.85002
[139]	validation_0-rmse:0.67423	validation_1-rmse:0.84931
[140]	validation_0-rmse:0.67242	validation_1-rmse:0.84864
[141]	validation_0-rmse:0.67066	validation_1-rmse:0.84796
[142]	validation_0-rmse:0.66893	validation_1-rmse:0.84735
[143]	validation_0-rmse:0.66730	validation_1-rmse:0.84675
[144]	validation_0-rmse:0.66554	validation_1-rmse:0.84609
[145]	validation_0-rmse:0.66401	validation_1-rmse:0.84553
[146]	validation_0-rmse:0.66233	validation_1-rmse:0.84497
[147]	validation_0-rmse:0.66075	validation_1-rmse:0.84434
[148]	validation_0-rmse:0.65910	validation_1-rmse:0.84377
[149]	validation_0-rmse:0.65734	validation_1-rmse:0.84313
[150]	validation_0-rmse:0.65577	validation_1-rmse:0.84258
[151]	validation_0-rmse:0.65428	validation_1-rmse:0.84204
[152]	validati

[277]	validation_0-rmse:0.53324	validation_1-rmse:0.81078
[278]	validation_0-rmse:0.53248	validation_1-rmse:0.81062
[279]	validation_0-rmse:0.53191	validation_1-rmse:0.81050
[280]	validation_0-rmse:0.53138	validation_1-rmse:0.81043
[281]	validation_0-rmse:0.53089	validation_1-rmse:0.81036
[282]	validation_0-rmse:0.53031	validation_1-rmse:0.81024
[283]	validation_0-rmse:0.52980	validation_1-rmse:0.81012
[284]	validation_0-rmse:0.52929	validation_1-rmse:0.81004
[285]	validation_0-rmse:0.52863	validation_1-rmse:0.80996
[286]	validation_0-rmse:0.52806	validation_1-rmse:0.80987
[287]	validation_0-rmse:0.52756	validation_1-rmse:0.80980
[288]	validation_0-rmse:0.52706	validation_1-rmse:0.80968
[289]	validation_0-rmse:0.52656	validation_1-rmse:0.80961
[290]	validation_0-rmse:0.52577	validation_1-rmse:0.80946
[291]	validation_0-rmse:0.52521	validation_1-rmse:0.80941
[292]	validation_0-rmse:0.52472	validation_1-rmse:0.80937
[293]	validation_0-rmse:0.52398	validation_1-rmse:0.80928
[294]	validati

[419]	validation_0-rmse:0.46304	validation_1-rmse:0.80039
[420]	validation_0-rmse:0.46275	validation_1-rmse:0.80036
[421]	validation_0-rmse:0.46249	validation_1-rmse:0.80034
[422]	validation_0-rmse:0.46221	validation_1-rmse:0.80033
[423]	validation_0-rmse:0.46179	validation_1-rmse:0.80024
[424]	validation_0-rmse:0.46151	validation_1-rmse:0.80022
[425]	validation_0-rmse:0.46108	validation_1-rmse:0.80013
[426]	validation_0-rmse:0.46083	validation_1-rmse:0.80012
[427]	validation_0-rmse:0.46068	validation_1-rmse:0.80007
[428]	validation_0-rmse:0.46039	validation_1-rmse:0.80000
[429]	validation_0-rmse:0.46021	validation_1-rmse:0.79996
[430]	validation_0-rmse:0.45985	validation_1-rmse:0.79990
[431]	validation_0-rmse:0.45961	validation_1-rmse:0.79987
[432]	validation_0-rmse:0.45926	validation_1-rmse:0.79982
[433]	validation_0-rmse:0.45913	validation_1-rmse:0.79979
[434]	validation_0-rmse:0.45887	validation_1-rmse:0.79975
[435]	validation_0-rmse:0.45853	validation_1-rmse:0.79970
[436]	validati

[561]	validation_0-rmse:0.42769	validation_1-rmse:0.79493
[562]	validation_0-rmse:0.42761	validation_1-rmse:0.79492
[563]	validation_0-rmse:0.42722	validation_1-rmse:0.79493
[564]	validation_0-rmse:0.42715	validation_1-rmse:0.79491
[565]	validation_0-rmse:0.42678	validation_1-rmse:0.79485
[566]	validation_0-rmse:0.42670	validation_1-rmse:0.79483
[567]	validation_0-rmse:0.42643	validation_1-rmse:0.79479
[568]	validation_0-rmse:0.42617	validation_1-rmse:0.79482
[569]	validation_0-rmse:0.42596	validation_1-rmse:0.79481
[570]	validation_0-rmse:0.42559	validation_1-rmse:0.79475
[571]	validation_0-rmse:0.42522	validation_1-rmse:0.79479
[572]	validation_0-rmse:0.42499	validation_1-rmse:0.79482
[573]	validation_0-rmse:0.42492	validation_1-rmse:0.79482
[574]	validation_0-rmse:0.42460	validation_1-rmse:0.79476
[575]	validation_0-rmse:0.42436	validation_1-rmse:0.79478
[576]	validation_0-rmse:0.42392	validation_1-rmse:0.79473
[577]	validation_0-rmse:0.42371	validation_1-rmse:0.79469
[578]	validati

...

[1961]	validation_0-rmse:0.23765	validation_1-rmse:0.78233
[1962]	validation_0-rmse:0.23752	validation_1-rmse:0.78230
[1963]	validation_0-rmse:0.23745	validation_1-rmse:0.78229
[1964]	validation_0-rmse:0.23742	validation_1-rmse:0.78229
[1965]	validation_0-rmse:0.23740	validation_1-rmse:0.78229
[1966]	validation_0-rmse:0.23728	validation_1-rmse:0.78227
[1967]	validation_0-rmse:0.23728	validation_1-rmse:0.78226
[1968]	validation_0-rmse:0.23714	validation_1-rmse:0.78224
[1969]	validation_0-rmse:0.23690	validation_1-rmse:0.78225
[1970]	validation_0-rmse:0.23679	validation_1-rmse:0.78226
[1971]	validation_0-rmse:0.23672	validation_1-rmse:0.78226
[1972]	validation_0-rmse:0.23663	validation_1-rmse:0.78225
[1973]	validation_0-rmse:0.23653	validation_1-rmse:0.78221
[1974]	validation_0-rmse:0.23646	validation_1-rmse:0.78222
[1975]	validation_0-rmse:0.23627	validation_1-rmse:0.78219
[1976]	validation_0-rmse:0.23626	validation_1-rmse:0.78219
[1977]	validation_0-rmse:0.23616	validation_1-rmse:0.782

[2100]	validation_0-rmse:0.22664	validation_1-rmse:0.78173
[2101]	validation_0-rmse:0.22663	validation_1-rmse:0.78173
[2102]	validation_0-rmse:0.22661	validation_1-rmse:0.78173
[2103]	validation_0-rmse:0.22657	validation_1-rmse:0.78174
[2104]	validation_0-rmse:0.22656	validation_1-rmse:0.78174
[2105]	validation_0-rmse:0.22652	validation_1-rmse:0.78175
[2106]	validation_0-rmse:0.22651	validation_1-rmse:0.78174
[2107]	validation_0-rmse:0.22642	validation_1-rmse:0.78174
[2108]	validation_0-rmse:0.22641	validation_1-rmse:0.78174
[2109]	validation_0-rmse:0.22638	validation_1-rmse:0.78173
[2110]	validation_0-rmse:0.22631	validation_1-rmse:0.78170
[2111]	validation_0-rmse:0.22621	validation_1-rmse:0.78170
[2112]	validation_0-rmse:0.22602	validation_1-rmse:0.78170
[2113]	validation_0-rmse:0.22595	validation_1-rmse:0.78168
[2114]	validation_0-rmse:0.22590	validation_1-rmse:0.78169
[2115]	validation_0-rmse:0.22583	validation_1-rmse:0.78169
[2116]	validation_0-rmse:0.22580	validation_1-rmse:0.781

[2239]	validation_0-rmse:0.21396	validation_1-rmse:0.78130
[2240]	validation_0-rmse:0.21387	validation_1-rmse:0.78131
[2241]	validation_0-rmse:0.21371	validation_1-rmse:0.78132
[2242]	validation_0-rmse:0.21351	validation_1-rmse:0.78135
[2243]	validation_0-rmse:0.21342	validation_1-rmse:0.78135
[2244]	validation_0-rmse:0.21338	validation_1-rmse:0.78136
[2245]	validation_0-rmse:0.21334	validation_1-rmse:0.78137
[2246]	validation_0-rmse:0.21327	validation_1-rmse:0.78137
[2247]	validation_0-rmse:0.21315	validation_1-rmse:0.78136
[2248]	validation_0-rmse:0.21311	validation_1-rmse:0.78137
[2249]	validation_0-rmse:0.21310	validation_1-rmse:0.78138
[2250]	validation_0-rmse:0.21297	validation_1-rmse:0.78136
[2251]	validation_0-rmse:0.21284	validation_1-rmse:0.78138
[2252]	validation_0-rmse:0.21275	validation_1-rmse:0.78138
[2253]	validation_0-rmse:0.21267	validation_1-rmse:0.78137
[2254]	validation_0-rmse:0.21254	validation_1-rmse:0.78137
[2255]	validation_0-rmse:0.21252	validation_1-rmse:0.781

[2378]	validation_0-rmse:0.20644	validation_1-rmse:0.78144
[2379]	validation_0-rmse:0.20620	validation_1-rmse:0.78145
[2380]	validation_0-rmse:0.20619	validation_1-rmse:0.78145
[2381]	validation_0-rmse:0.20613	validation_1-rmse:0.78146
[2382]	validation_0-rmse:0.20613	validation_1-rmse:0.78146
[2383]	validation_0-rmse:0.20607	validation_1-rmse:0.78144
[2384]	validation_0-rmse:0.20603	validation_1-rmse:0.78146
[2385]	validation_0-rmse:0.20580	validation_1-rmse:0.78143
[2386]	validation_0-rmse:0.20577	validation_1-rmse:0.78144
[2387]	validation_0-rmse:0.20576	validation_1-rmse:0.78144
[2388]	validation_0-rmse:0.20573	validation_1-rmse:0.78142
[2389]	validation_0-rmse:0.20569	validation_1-rmse:0.78143
[2390]	validation_0-rmse:0.20557	validation_1-rmse:0.78144
[2391]	validation_0-rmse:0.20551	validation_1-rmse:0.78144
[2392]	validation_0-rmse:0.20530	validation_1-rmse:0.78143
[2393]	validation_0-rmse:0.20524	validation_1-rmse:0.78143
[2394]	validation_0-rmse:0.20521	validation_1-rmse:0.781

XGBRegressor(base_score=0.7, booster='gbtree', boosting_type='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=12, max_features='auto', min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
#making prediction on the other part of the test data

pred3 = xg.predict(x_test2)
mean_squared_error(pred3, y_test2) ** 0.5

0.7811811448965625

In [8]:
pred3 = scalery.inverse_transform(pred3)
mean_squared_error(pred3, y_test2) ** 0.5

12.382449088527002

In [14]:
#eval1 = cross_val_score(xg, train_enc, labels, scoring='neg_mean_squared_error', cv=5)
#print((eval1.mean() * -1 ) ** 0.5)  #7 max depth

In [7]:
#preparing the predictions make into the sample file for submission

preds1 = xg.predict(test_enc)
sample1 = pd.read_csv('SampleSubmission.csv')
test = pd.read_csv('test.csv')
sample1.id = test.id
sample1.target = preds1
sample1.to_csv('XG6.csv', index = False)

In [16]:
params = {
    'max_depth': [10, 12, 14]
}

xg1 = xgb.XGBRegressor()

clf = GridSearchCV(xg, param_grid=params, verbose=2, cv=2)

In [17]:
clf.fit(x_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] max_depth=10 ....................................................
[12:12:31] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..................................... max_depth=10, total=34.2min
[CV] max_depth=10 ....................................................
[12:46:41] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 34.2min remaining:    0.0s


KeyboardInterrupt: 